
<hr style="border: 2px solid #4CAF50; width: 100%;">

<h1 style="text-align:center; font-family: 'bold', sans-serif; color:#4CAF50;">
    🚀 1. Dependencies 👾
</h1>

<hr style="border: 2px solid #4CAF50; width: 100%;">



In [2]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import evaluate
import gc

In [3]:
# Connect to GPU (make sure you've selected GPU in Colab)
assert torch.cuda.is_available(), "Change runtime type to GPU!"
device = torch.device("cuda")
print(f"Using device: {device}")

Using device: cuda


In [4]:
dataset = load_dataset("Jinyan1/COLING_2025_MGT_en")
print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/588 [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/287M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/286M [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/246M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/610767 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/261758 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'source', 'sub_source', 'lang', 'model', 'label', 'text'],
        num_rows: 610767
    })
    dev: Dataset({
        features: ['id', 'source', 'sub_source', 'lang', 'model', 'label', 'text'],
        num_rows: 261758
    })
})



<hr style="border: 2px solid #4CAF50; width: 100%;">

<h1 style="text-align:center; font-family: 'bold', sans-serif; color:#4CAF50;">
    🚀 2. EDA 👾
</h1>

<hr style="border: 2px solid #4CAF50; width: 100%;">



In [5]:
# Exploratory Data Analysis
print("\n Train Class distribution:")
dataset["train"].to_pandas()["label"].value_counts()



 Train Class distribution:


,count
label,
1,381845
0,228922


In [6]:
print("\n Validation Class distribution:")
dataset["dev"].to_pandas()["label"].value_counts()


 Validation Class distribution:


,count
label,
1,163430
0,98328


In [7]:
def label_categories(model_name):
    """Maps model names to their respective companies/categories"""
    category_map = {
        # Human-written content
        'human': 'human',

        # OpenAI Models
        'gpt-3.5-turbo': 'openai',
        'davinci': 'openai',
        'gpt-35': 'openai',
        'text-davinci-003': 'openai',
        'text-davinci-002': 'openai',
        'gpt4': 'openai',
        'gpt4o': 'openai',

        # Meta (Facebook) Models
        'llama3-70b': 'meta',
        'llama3-8b': 'meta',

        # Google Models
        'gemma-7b-it': 'google',
        'gemma2-9b-it': 'google',
        'flan_t5_small': 'google',
        'flan_t5_base': 'google',
        'flan_t5_xxl': 'google',
        'flan_t5_large': 'google',
        'flan_t5_xl': 'google',

        # BigScience Models
        'bloomz': 'bigscience',
        'bloom_7b': 'bigscience',

        # Cohere Models
        'cohere': 'cohere',

        # Databricks Models
        'dolly': 'databricks',
        'dolly-v2-12b': 'databricks',

        # Mistral Models
        'mixtral-8x7b': 'mistral',

        # BigScience T0 Models
        't0_3b': 'bigscience_t0',
        't0_11b': 'bigscience_t0',

        # Facebook OPT Models
        'opt_iml_max_1.3b': 'facebook_opt',
        'opt_iml_30b': 'facebook_opt',
        'opt_2.7b': 'facebook_opt',
        'opt_125m': 'facebook_opt',
        'opt_6.7b': 'facebook_opt',
        'opt_30b': 'facebook_opt',
        'opt_1.3b': 'facebook_opt',
        'opt_350m': 'facebook_opt',
        'opt_13b': 'facebook_opt',

        # Other Models
        '65B': 'other',
        '30B': 'other',
        '13B': 'other',
        '7B': 'other',
        'gpt_neox': 'other',
        'GLM130B': 'other',
        'gpt_j': 'other'
    }
    return category_map.get(model_name, 'unknown')

In [8]:
from datasets import Dataset

def add_category_column(example):
    example['category'] = label_categories(example['model'])
    return example

# Apply to HuggingFace dataset
dataset = dataset.map(add_category_column)

# For train/dev/test splits
train_dataset = dataset['train'].map(add_category_column)
dev_dataset = dataset['dev'].map(add_category_column)

Map:   0%|          | 0/610767 [00:00<?, ? examples/s]

Map:   0%|          | 0/261758 [00:00<?, ? examples/s]

Map:   0%|          | 0/610767 [00:00<?, ? examples/s]

Map:   0%|          | 0/261758 [00:00<?, ? examples/s]

In [9]:
train_df = train_dataset.to_pandas()
dev_df = dev_dataset.to_pandas()

In [10]:
print(f"Train size: {len(train_df)}")
print(f"Validation size: {len(dev_df)}")

Train size: 610767
Validation size: 261758


In [11]:
train_df['category'].value_counts()

,count
category,
human,228922
openai,132409
google,53125
facebook_opt,50603
other,34652
bigscience,26212
meta,24390
cohere,20808
mistral,15865


In [12]:
human = (train_df['label'] == 0).sum()
ai = (train_df['label'] == 1).sum()

print(f"Human: {human}")
print(f"AI: {ai}")

Human: 228922
AI: 381845


In [13]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


label_counts = {0: human, 1: ai}
model_counts = dev_df['category'].value_counts()
model_counts = model_counts[model_counts.index != 'human']# exclude human
model_counts = model_counts.head(7)
model_counts = model_counts.to_dict()


fig = make_subplots(rows=1, cols=2, specs=[[{"type": "bar"}, {"type": "pie"}]], subplot_titles=("Label Distribution", "Model Distribution"))


# First subplot: Label distribution
fig.add_trace(go.Bar(x=list(label_counts.keys()), y=list(label_counts.values()), marker_color=['#E91E63', '#FF9800']), row=1, col=1)
fig.update_xaxes(title_text="Labels", row=1, col=1)
fig.update_yaxes(title_text="Count", row=1, col=1)


# Second subplot: Model distribution (Pie chart with BUPU palette)
colors = ['#E91E63', '#FF9800', '#FFEB3B', '#4CAF50', '#2196F3', '#9C27B0'] # Example BUPU colors, extend as needed
fig.add_trace(go.Pie(labels=list(model_counts.keys()), values=list(model_counts.values()), marker_colors=colors[:len(model_counts)]), row=1, col=2)


fig.update_layout(title_text="Data Distribution", showlegend=True, height=500, width=1000)
fig.show()



<hr style="border: 2px solid #4CAF50; width: 100%;">

<h1 style="text-align:center; font-family: 'bold', sans-serif; color:#4CAF50;">
    🚀 3. Text Processing 👾
</h1>

<hr style="border: 2px solid #4CAF50; width: 100%;">



In [14]:
import re
before_processing =  train_df['text'][5]
def safe_preprocess(text):
    """
    Minimal preprocessing that preserves AI detection signals
    """
    if not isinstance(text, str):
        return ""

    # 1. Basic normalization (preserves linguistic patterns)
    text = text.lower()
    text = re.sub(r'\n+', ' [NEWLINE] ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # 2. Only remove artifacts that add noise without signal
    text = re.sub(r'http\S+|www\S+|https\S+', '[URL]', text)  # Replace URLs

    return text

# Apply to dataframe
train_df['text'] = train_df['text'].apply(safe_preprocess)
dev_df['text'] = dev_df['text'].apply(safe_preprocess)

# Verify with sample
sample_idx = 5
print("Original:\n", before_processing)
print("\nProcessed:\n", train_df['text'].iloc[sample_idx])

Original:
 Offshore tax shelters are financial arrangements that are set up in countries other than the one where the individual or company resides, with the goal of reducing or eliminating tax liability. There are several ways in which wealthy individuals or companies can move their money offshore without getting taxed on it, but it's important to note that these activities may be illegal or unethical, depending on the specific circumstances and the laws of the countries involved.\n\nOne way to move money offshore is to set up a company or trust in a country with favorable tax laws, such as a low tax rate or tax-free status. This company or trust can then be used to hold financial assets, such as stocks, bonds, or real estate, and the income generated from these assets may not be subject to tax in the country where the individual or company resides.\n\nAnother way to move money offshore is to use financial instruments, such as offshore bank accounts or shell corporations, to conceal t